In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import string
import nltk
from nltk.corpus import stopwords

In [2]:
# Asegúrate de descargar stopwords si no lo has hecho antes
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# ArCo DNP
#arco = pd.read_excel('C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/2024 11 01 - Base Original Desc Larga.xlsx')

# ArCo Remoto
arco = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/2024 11 01 - Base Original.xlsx')

In [4]:
arco.head(100)

,Código,Sector,Entidad,Nombre del Instrumento,Descripción,Página Web,Antigüedad,Iteración,Emprendedores,Mipymes,...,1. Señale cuál o cuáles de las siguientes brechas,2. ¿En el diseño del instrumento se analizó el posible impacto diferenciado?,3. ¿El instrumento incorpora un enfoque interseccional?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro,Estado
0,1794,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Plan de Educación Financiera,A partir del modelo pedagógico de la entidad p...,https://www.colpensiones.gov.co/educacion/,5.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
1,1800,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Programa Beps,Los requisitos para vincularse a Hoy y Mañana ...,https://www.colpensiones.gov.co/beps/,9.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
2,1797,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Silverexpo. Feria de entretenimiento y bienest...,La feria Silverexpo se realiza en alianza con ...,www.colpensiones.gov.co,4.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
3,3689,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Adecuación de tierras,La adecuación de tierras es el servicio públic...,https://www.adr.gov.co/atencion-y-servicios-a-...,8.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,Si,No,Si,116422783387,0,0,0,0,0,Actualizado
4,1734,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Fortalecimiento a la prestación del servicio p...,MEJORAR LA TRANSFERENCIA Y APROPIACIÓN DEL CON...,https://www.adr.gov.co/atencion-y-servicios-a-...,4.0,Primera iteración 2024 - Mes de reporte: Septi...,No,No,...,No,No,No,122305692809,0,0,0,0,0,Desactualizado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2426,Planeación,Departamento Nacional de Planeación - DNP,Caja de Herramientas para intervenciones resil...,"Con el apoyo de MinAmbiente, MinHacienda y la ...",https://2022.dnp.gov.co/programas/ambiente/ges...,6.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,0,0,0,0,0,0,Actualizado
96,2445,Planeación,Departamento Nacional de Planeación - DNP,Corredor de financiamiento climático,Para acelerar en la construcción de un modelo ...,https://finanzasdelclima.dnp.gov.co/iniciativa...,0.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,0,0,0,0,0,0,Actualizado
97,2430,Planeación,Departamento Nacional de Planeación - DNP,Curso Finanzas del Clima,El Departamento Nacional de Planeación como lí...,https://finanzasdelclima.dnp.gov.co/movilizaci...,2.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,0,0,0,0,0,0,Actualizado
98,2429,Planeación,Departamento Nacional de Planeación - DNP,Curso virtual sobre crecimiento verde,El modelo de crecimiento verde ha sido impleme...,https://campus.dnp.gov.co/,2.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,0,0,0,0,0,0,Actualizado


In [5]:
# 2. Definir categorías
categorias = [
    "Desarrollo de capacidades básicas",
    "Transferencia de tecnología",
    "Educación y formación",
    "Infraestructura básica",
    "Fortalecimiento de capacidades intermedias",
    "Incentivos a la innovación",
    "Colaboración público-privada",
    "Protección de la propiedad intelectual",
    "Capacidades avanzadas de innovación",
    "Investigación y desarrollo (I+D) avanzada",
    "Redes de innovación globales",
    "Políticas de innovación integradas",
]

In [6]:
# 3. Preprocesamiento de texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]  # Eliminar stopwords
    return " ".join(palabras)

In [30]:
arco["Descripción_limpia"] = arco["Descripción"].apply(lambda x: limpiar_texto(str(x)))



In [31]:
# 4. Vectorización TF-IDF
vectorizer = TfidfVectorizer()
tfidf_descripciones = vectorizer.fit_transform(arco["Descripción_limpia"])
tfidf_categorias = vectorizer.transform(categorias)

In [32]:
# 5. Calcular similitud de coseno
similitudes = cosine_similarity(tfidf_descripciones, tfidf_categorias)

In [33]:
# 6. Asignación de categorías
umbral = 0.2  # Define un umbral para determinar la similitud mínima
for i, cat in enumerate(categorias):
    arco[cat] = np.where(similitudes[:, i] > umbral, "Sí", "No") 

In [34]:
# 7. Asignar "Otra" si ninguna categoría es relevante
arco["Otra"] = np.where(arco[categorias].eq("Sí").any(axis=1), "", "Otra")

In [35]:
# 8. Guardar el resultado en un nuevo archivo Excel
output_file = "resultado_categorización.xlsx"
arco.to_excel(output_file, index=False)
print(f"El archivo clasificado se ha guardado como '{output_file}'.")

El archivo clasificado se ha guardado como 'resultado_categorización.xlsx'.
